In [58]:
!pip install pytorch_lightning

2716.73s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 29.9 MB/s eta 0:00:00


In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from tokenizers import Tokenizer, trainers, models, pre_tokenizers, normalizers
from datasets import load_dataset
from datasets import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math
import json
import os

torch.manual_seed(1223)

mps_device = torch.device("mps")

In [ ]:
# Load the English-German subset of the WMT 2014 dataset
dataset = load_dataset("wmt14", "de-en")

# Explore the dataset
dataset

In [ ]:
def save_to_json(dataset_split, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        # Convert dataset to a list of dictionaries and save to JSON
        json.dump([example for example in dataset_split], f, ensure_ascii=False, indent=4)
        
def load_from_json(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return Dataset.from_list(data)


In [ ]:
def save_to_json(dataset_split, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        # Convert dataset to a list of dictionaries and save to JSON
        json.dump([example for example in dataset_split], f, ensure_ascii=False, indent=4)


# save train, validation, or test splits locally in json format
save_to_json(dataset['train'], 'train_data.json')
save_to_json(dataset['validation'], 'validation_data.json')
save_to_json(dataset['test'], 'test_data.json')


In [3]:
# load train, validation and test splits
train_dataset = load_from_json('train_data.json')
validation_dataset = load_from_json('validation_data.json')
test_dataset = load_from_json('test_data.json')

In [4]:
print("Datasets imported from JSON files.")
# Print first 3 examples from re-imported train dataset
for i in range(3):
    print(f"Example {i}:")
    print(f"  German: {train_dataset[i]['translation']['de']}")
    print(f"  English: {train_dataset[i]['translation']['en']}")

Datasets imported from JSON files.
Example 0:
  German: Wiederaufnahme der Sitzungsperiode
  English: Resumption of the session
Example 1:
  German: Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.
  English: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Example 2:
  German: Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.
  English: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.


In [5]:
train_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['translation'],
     num_rows: 4508785
 }),
 Dataset({
     features: ['translation'],
     num_rows: 3000
 }),
 Dataset({
     features: ['translation'],
     num_rows: 3003
 }))

In [ ]:
# can be used for training separate tokenizers
# don't use for a single tokenizer
os.makedirs("tokenizer_data", exist_ok=True)

with open("tokenizer_data/english.txt", "w", encoding="utf-8") as eng_file, \
     open("tokenizer_data/german.txt", "w", encoding="utf-8") as ger_file:
    
    for split in ['train', 'validation', 'test']:
        for example in dataset[split]:
            eng_file.write(example['translation']['en'] + "\n")
            ger_file.write(example['translation']['de'] + "\n")

In [ ]:
with open("tokenizer_data/common.txt", "w", encoding="utf-8") as common_file:
    
    for split in ['train', 'validation', 'test']:
        for example in dataset[split]:
            common_file.write(example['translation']['en'] + "\n")
            common_file.write(example['translation']['de'] + "\n")

In [6]:
allowed_sequence_length=1000


In [7]:

tokenizer = Tokenizer(models.BPE(unk_token='[UNK]'))

tokenizer.normalizer = normalizers.Sequence([
     normalizers.Lowercase()
])

tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
     pre_tokenizers.Whitespace(),
])

tokenizer.enable_truncation(max_length=allowed_sequence_length)

trainer = trainers.BpeTrainer(vocab_size=37000, special_tokens=["[UNK]", "[START]", "[END]", "[MASK]"], show_progress=True)

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
tokenizer.train(["tokenizer_data/common.txt"], trainer=trainer)

In [ ]:
tokenizer.save('bpe2.json')

In [8]:
tokenizer = Tokenizer.from_file('bpe2.json')

In [9]:
tokenizer.get_vocab_size()

37000

In [10]:
stoi = tokenizer.get_vocab()
itos = { i:s for s, i in stoi.items() }
list(itos.items())[:10], list(stoi.items())[:10]

([(7695, 'bleibt'),
  (7035, 'establish'),
  (27362, 'vaters'),
  (25984, 'steering'),
  (7204, 'acy'),
  (22122, 'certificates'),
  (27713, 'zugriffs'),
  (315, '̃'),
  (14920, 'plural'),
  (18637, 'contractual')],
 [('bleibt', 7695),
  ('establish', 7035),
  ('vaters', 27362),
  ('steering', 25984),
  ('acy', 7204),
  ('certificates', 22122),
  ('zugriffs', 27713),
  ('̃', 315),
  ('plural', 14920),
  ('contractual', 18637)])

In [11]:
itos[0], itos[1], itos[2], itos[3], itos[4]

('[UNK]', '[START]', '[END]', '[MASK]', '!')

In [12]:
ger_sent = train_dataset[0]['translation']['de']
eng_sent = train_dataset[0]['translation']['en']

ger_enc = tokenizer.encode(ger_sent)
eng_enc = tokenizer.encode(eng_sent)

print(eng_sent)
print(f'Tokens: {eng_enc.tokens}')
print(f'Ids: {eng_enc.ids}')
print('------')
print(ger_sent)
print(f'Tokens: {ger_enc.tokens}')
print(f'Ids: {ger_enc.ids}')

Resumption of the session
Tokens: ['resumption', 'of', 'the', 'session']
Ids: [34500, 3600, 3581, 8962]
------
Wiederaufnahme der Sitzungsperiode
Tokens: ['wiederaufnahme', 'der', 'sitzungsperiode']
Ids: [24924, 3588, 24101]


In [ ]:
# longest sentence over 14000 tokens long
eng_sentences = [ example['translation']['en'] for example in train_dataset ]
ger_sentences = [ example['translation']['de'] for example in train_dataset ]
max_length = max(max(len(tokenizer.encode(eng).tokens), len(tokenizer.encode(ger).tokens)) for eng, ger in zip(eng_sentences, ger_sentences))
print(f'Longest sentence: {max_length} words')

In [31]:
# calculating the positional encoding

def gen_pos_enc(seq_len, d_model, n):
    pos_enc = torch.zeros((seq_len, d_model), dtype=torch.float)
    for k in range(seq_len):
        for i in range(d_model//2):
            theta = k / (n ** (2*i / d_model))
            pos_enc[k, 2*i] = math.sin(theta)
            pos_enc[k, 2*i+1] = math.cos(theta)
            
    pos_enc.requires_grad=False
    return pos_enc

pos_enc = gen_pos_enc(allowed_sequence_length, 512, 10000)
pos_enc.shape, pos_enc.device

(torch.Size([1000, 512]), device(type='cpu'))

In [32]:
pos_enc = pos_enc.to(mps_device)
pos_enc.device

device(type='mps', index=0)

In [ ]:
# Custom collate function for tokenization and padding
def collate_fn(batch):
    tokenized_batch = []
    for example in batch:
        # Tokenize and encode sentences
        eng = tokenizer.encode(example["translation"]["en"]).ids
        ger = tokenizer.encode(example["translation"]["de"]).ids
        
        tokenized_batch.append((eng, ger))
    
    # Find the maximum sequence length in the batch
    max_eng_length = max(len(eng) for (eng, _) in tokenized_batch)
    max_ger_length = max(len(ger) for (_, ger) in tokenized_batch)

    # Pad all sequences to the maximum length
    sources = [[1] + eng + [2] * (max_eng_length - len(eng)) + [2] for eng, _ in tokenized_batch]
    targets = [[1] + ger + [2] * (max_ger_length - len(ger)) + [2] for _, ger in tokenized_batch]
    
    batch = {
        "sources": torch.tensor(sources, dtype=torch.long), 
        "targets": torch.tensor(targets, dtype=torch.long)
    }
    
    # Convert to PyTorch tensor
    return batch

# Create dataloaders
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True, 
    collate_fn=collate_fn
)

# Example: Iterate through the DataLoader
for batch in train_dataloader:
    print("Sources shape:", batch['sources'].shape)  # Shape: (batch_size, max_sequence_length)
    print("Targets shape:", batch['targets'].shape)  # Shape: (batch_size, max_sequence_length)
    print(batch)  
    break

Sources shape: torch.Size([64, 92])
Targets shape: torch.Size([64, 74])
{'sources': tensor([[    1,  3878,    15,  ...,     2,     2,     2],
        [    1,  3581,  7862,  ...,     2,     2,     2],
        [    1,  3581,  5571,  ...,     2,     2,     2],
        ...,
        [    1,  4805,  3809,  ...,     2,     2,     2],
        [    1,  3581, 10254,  ...,     2,     2,     2],
        [    1,    42, 27071,  ...,     2,     2,     2]]), 'targets': tensor([[    1,  3757,  3641,  ...,     2,     2,     2],
        [    1,  3605,  7862,  ...,     2,     2,     2],
        [    1,  3605,  6586,  ...,     2,     2,     2],
        ...,
        [    1, 18254,  3651,  ...,  3760,    17,     2],
        [    1,  4980,  3605,  ...,     2,     2,     2],
        [    1,  3599, 27071,  ...,     2,     2,     2]])}


In [ ]:
class Module(nn.Module):
    device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))


class FeedForwardNetword(Module):
    
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear_in = nn.Linear(d_model, d_ff)
        self.activation = nn.ReLU()
        self.linear_out = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.linear_in(x)
        x = self.activation(x)
        x = self.linear_out(x)
        return x


# the following implementation of Multi Head Attention is slow and unsuitable for training 
class ScaledDotProductAttention(Module):
    
    def __init__(self, d_model, head_size, use_mask=False):
        super().__init__()
        self.head_size = head_size
        self.w_q = nn.Linear(d_model, head_size, bias=False) # (batch_size, head_size)
        self.w_k = nn.Linear(d_model, head_size, bias=False)
        self.w_v = nn.Linear(d_model, head_size, bias=False)
        self.use_mask = use_mask        
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, q, k, v):
        q = self.w_q(q) # (batch_size, sec_len, d_model) @ (d_model, head_size) = (batch_size, sec_len, head_size)
        k = self.w_k(k) 
        v = self.w_v(v)
        
        rel = q @ k.transpose(-2, -1) # (batch_size, sec_length, sec_length)
        rel = rel * self.head_size**-0.5 
        
        if self.use_mask:
            sec_len = rel.size(-1)
            mask = torch.tril(torch.ones(sec_len, sec_len, requires_grad=False,)).to(self.device)
            rel = rel.masked_fill(mask == 0, float('-inf'))
        
        value_weights = self.softmax(rel)
        
        return value_weights @ v # (batch_size, sec_length, sec_length) @ ()

        
class MultiHeadAttention(Module):
    
    def __init__(self, d_model, num_heads, use_mask=False):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.use_mask = use_mask
        
        head_size = d_model // num_heads
        self.heads = nn.ModuleList([ScaledDotProductAttention(d_model, head_size, use_mask) for _ in range(num_heads)])
        self.w_o = nn.Linear(d_model, d_model, bias=False)
    
    def forward(self, q, k, v):
        merged = torch.cat([head(q, k, v) for head in self.heads], dim=-1)
        return self.w_o(merged)


class EncoderLayer(Module):
    
    def __init__(self, d_model, d_ff, num_heads, dropout_rate):
        super().__init__()
        self.encoderLayers = []
            
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.dropout_1 = nn.Dropout(p=dropout_rate)
        self.layer_norm_1 = nn.LayerNorm(d_model)
            
        self.ffn = FeedForwardNetword(d_model, d_ff)
        self.dropout_2 = nn.Dropout(p=dropout_rate)
        self.layer_norm_2 = nn.LayerNorm(d_model)
    
    def forward(self, x):
        att_out = self.attention(x, x, x)
        x = self.layer_norm_1(x + self.dropout_1(att_out))
        
        ffn_out = self.ffn(x)
        x = self.layer_norm_2(x + self.dropout_2(ffn_out))
        return x


class DecoderLayer(Module):
    
    def __init__(self, d_model, d_ff, num_heads, dropout_rate):
        super().__init__()
        
        self.attention = MultiHeadAttention(d_model, num_heads, use_mask=True)
        self.dropout_1 = nn.Dropout(p=dropout_rate)
        self.layer_norm_1 = nn.LayerNorm(d_model)
        
        self.enc_attention = MultiHeadAttention(d_model, num_heads)
        self.dropout_2 = nn.Dropout(p=dropout_rate)
        self.layer_norm_2 = nn.LayerNorm(d_model)
            
        self.ffn = FeedForwardNetword(d_model, d_ff)
        self.dropout_3 = nn.Dropout(p=dropout_rate)
        self.layer_norm_3 = nn.LayerNorm(d_model)
    
    def forward(self, x, enc_out):
        att_out = self.attention(x, x, x)
        x = self.layer_norm_1(x + self.dropout_1(att_out))
        
        enc_att_out = self.enc_attention(q=x, k=enc_out, v=enc_out)
        x = self.layer_norm_2(x + self.dropout_2(enc_att_out))
        
        fnn_out = self.ffn(x)
        x = self.layer_norm_3(x + self.dropout_3(fnn_out))
        return x


class Embedding(Module):
        
    def __init__(self, vocab_size, d_model, pos_enc, dropout_rate=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model, 2) # 2 is the eos token, we also pad with it
        self.dropout = nn.Dropout(dropout_rate)
        self.pos_enc = pos_enc
            
    def forward(self, x):
        emb_out = self.embedding(x) # (batch, sequence, embedding)
        sec_len = emb_out.shape[1]
        return self.dropout(emb_out + self.pos_enc[:sec_len,:])
    

class Transformer(Module):
        
    def __init__(self, vocab_size, d_model, d_ff, pos_enc, num_heads=8, N=6, seed=5012025, dropout_rate=0.1):
        super().__init__()
        torch.manual_seed(seed)
        self.source_embedding = Embedding(vocab_size, d_model, pos_enc, dropout_rate=dropout_rate) 
        self.target_embedding = Embedding(vocab_size, d_model, pos_enc, dropout_rate=dropout_rate)
        

        self.encoder_stack = nn.ModuleList([EncoderLayer(d_model, d_ff, num_heads, dropout_rate) for _ in range(N)])
        self.decoder_stack = nn.ModuleList([DecoderLayer(d_model, d_ff, num_heads, dropout_rate) for _ in range(N)])
            
        self.linear = nn.Linear(d_model, vocab_size)
            
        self.to(self.device)
            
        torch.seed()
            
    def forward(self, source, target):
        
        enc_out = self.source_embedding(source)
        for encoder_layer in self.encoder_stack:
            enc_out = encoder_layer(enc_out)
            
        dec_out = self.target_embedding(target)
        for decoder_layer in self.decoder_stack:
            dec_out = decoder_layer(dec_out, enc_out)
            
        return self.linear(dec_out)

In [ ]:
def train_epoch(model: Transformer, train_dataloader, optimizer, loss_function):
    model.train()
    train_loss = 0
    num_batches = 0

    for batch in train_dataloader:
        sources = batch['sources'].to(model.device)
        targets = batch['targets'].to(model.device)
        
        optimizer.zero_grad()
        predictions = model(sources, targets[:, :-1]) 
        
        B, S, C = predictions.shape
        
        loss = loss_function(predictions.reshape(-1, C), targets[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        
        print(f'training loss: {loss.item()}')
        train_loss += loss.item()
        num_batches += 1
        
        if num_batches == 10:
            break


model = Transformer(37000, 512, 2048, pos_enc, 8, 6)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=1e-9)
loss_function = torch.nn.CrossEntropyLoss()
sheduler = ReduceLROnPlateau(optimizer, factor=0.5)



train_epoch(model, train_dataloader, optimizer, loss_function)
            

training loss: 10.383936882019043
training loss: 9.614827156066895
training loss: 8.692097663879395
training loss: 7.98657751083374
training loss: 7.713242530822754
training loss: 7.339689254760742
training loss: 6.295281410217285
training loss: 5.978188514709473
training loss: 5.56548547744751
training loss: 4.410912990570068


In [61]:
for param in model.parameters():
    print(param.shape)

torch.Size([37000, 512])
torch.Size([37000, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])


AttributeError: 'Transformer' object has no attribute 'example_input_array'